In [1]:
import os

In [2]:
%pwd

'd:\\Angus_Issues\\NorthCarolina_CameroonChapter_AngusIssues\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import joblib

In [5]:
@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    order: tuple
    target_column: float

In [6]:
# Assuming the presence of helper functions and constants
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH 
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ARIMA
        schema = self.schema
        target_column = list(self.schema['TARGET_COLUMN'].keys())[0]

        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_data_path=Path(config.train_data_path),
            test_data_path=Path(config.test_data_path),
            model_name=config.model_name,
            order=(params.p, params.d, params.q),
            target_column=target_column
        )

        return model_trainer_config

In [12]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        model = ARIMA(train_y, order=self.config.order)
        model_fit = model.fit()

        joblib.dump(model_fit, os.path.join(self.config.root_dir, self.config.model_name))

In [13]:
# Main training execution
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-08-08 12:39:31,265: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-08 12:39:31,268: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-08 12:39:31,272: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-08 12:39:31,273: INFO: common: created directory at: artifacts]
